In [ ]:
!pip install pystac-client

In [ ]:
import pystac_client
import planetary_computer

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
bbox_of_interest = [130.1336597270651,47.20167448654658,130.46158452041874,47.42586974012832]
time_of_interest = "2016-06-01/2016-06-05"

In [ ]:
search = catalog.search(
     collections=["landsat-c2-l2"],
     bbox=bbox_of_interest,
     datetime=time_of_interest,
     query={"eo:cloud_cover": {"lt": 10}},
 )
items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
from pystac.extensions.eo import EOExtension as eo
selected_item = min(items, key=lambda item: eo.ext(item).cloud_cover)
print(
     f"Choosing {selected_item.id} from {selected_item.datetime.date()}"
     + f" with {selected_item.properties['eo:cloud_cover']}% cloud cover"
)

In [ ]:
for key, asset in selected_item.assets.items():
     print(key, asset)

In [ ]:
from IPython.display import Image
Image(url=selected_item.assets["rendered_preview"].href)

In [ ]:
from osgeo import gdal
import numpy as np

#  读取tif数据集
def read_img(fileName, xoff=0, yoff=0, data_width=0, data_height=0):
    dataset = gdal.Open(fileName)
    if dataset == None:
        print(fileName + "文件无法打开")
    #  栅格矩阵的列数
    width = dataset.RasterXSize
    #  栅格矩阵的行数
    height = dataset.RasterYSize
    #  波段数
    bands = dataset.RasterCount
    #  获取数据
    if (data_width == 0 and data_height == 0):
        data_width = width
        data_height = height
    data = dataset.ReadAsArray(xoff, yoff, data_width, data_height)
    #  获取仿射矩阵信息
    geotrans = dataset.GetGeoTransform()
    #  获取投影信息
    proj = dataset.GetProjection()
    return width, height, bands, data, geotrans, proj

def write_img(filename, im_geotrans, im_proj, im_data):
    if 'int8' in im_data.dtype.name:
        datatype = gdal.GDT_Byte
    elif 'int16' in im_data.dtype.name:
        datatype = gdal.GDT_UInt16
    else:
        datatype = gdal.GDT_Float32

    if len(im_data.shape) == 3:
        im_bands, im_height, im_width = im_data.shape
    else:
        im_bands, (im_height, im_width) = 1, im_data.shape

    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(filename, im_width, im_height, im_bands, datatype, options=["TILED=YES", "COMPRESS=LZW"])

    dataset.SetGeoTransform(im_geotrans)
    dataset.SetProjection(im_proj)
    if im_bands == 1:
        dataset.GetRasterBand(1).WriteArray(im_data)
    else:
        for i in range(im_bands):
            dataset.GetRasterBand(i + 1).WriteArray(im_data[i])

    del dataset

band_list = ["red", "green", "blue"]
array_list, img_geotrans, img_proj = [], (0, 0, 0, 0, 0, 0), ""
for band_name in band_list:    
    width, height, bands, data, geotrans, proj = read_img(selected_item.assets[band_name].href)
    img_geotrans = geotrans
    img_proj = proj
    array_list.append(data)
array_list = np.array(array_list)
write_img("landsat8_SR.tif", img_geotrans, img_proj, array_list)
print("raster download finished!")